# 🎭 Cosplay AI Generator - Imagen 4 Ultra

**Simple, all-in-one cosplay transformation using Google's Imagen 4 Ultra**

## Setup:
1. Put your model photos in `models/` folder
2. Configure `.env` with your Google Cloud credentials
3. Run the cells below
4. Results will be saved in `output/` folder


In [1]:
# 📦 Install required packages
!pip install google-cloud-aiplatform python-dotenv pillow requests

  Using cached google_cloud_aiplatform-1.117.0-py2.py3-none-any.whl.metadata (41 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_bigquery-3.38.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached google_cloud_resource_manager-1.14.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached shapely-2.1.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached google_genai-1.39.1-py3-none-any.whl.metadata (45 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached grpc_google_iam_v1-0.14.2-py3-none-any.whl.metadata (9.1 kB)
  Using cached google_crc32c-1.7.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (2.3 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
Using cached google_cloud_aiplatform-1.117.0-py2.py3-n

In [1]:
# 🔧 Import all dependencies
import os
import json
import base64
import time
from datetime import datetime
from pathlib import Path
from PIL import Image
import requests
from dotenv import load_dotenv
from google.auth import default
from google.auth.transport.requests import Request

# Load environment variables
load_dotenv()

print("✅ All packages imported successfully!")

✅ All packages imported successfully!


In [2]:
# 🛠️ Configuration
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT_ID")
LOCATION = os.getenv("IMAGEN_LOCATION", "us-central1")
CREDENTIALS_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# Folders
MODELS_FOLDER = Path("models")
OUTPUT_FOLDER = Path("output")
CHARACTERS_FILE = Path("characters.json")

# Create folders if they don't exist
MODELS_FOLDER.mkdir(exist_ok=True)
OUTPUT_FOLDER.mkdir(exist_ok=True)

print(f"📁 Models folder: {MODELS_FOLDER.absolute()}")
print(f"📁 Output folder: {OUTPUT_FOLDER.absolute()}")
print(f"🚀 Project ID: {PROJECT_ID}")
print(f"🌍 Location: {LOCATION}")

📁 Models folder: /Users/yaga/CosplayAI/cosplay-simple/models
📁 Output folder: /Users/yaga/CosplayAI/cosplay-simple/output
🚀 Project ID: cosplayai
🌍 Location: us-central1


In [3]:
# 🎭 Load character definitions
with open(CHARACTERS_FILE, 'r') as f:
    CHARACTERS = json.load(f)

print("Available characters:")
for char_id, char_data in CHARACTERS.items():
    print(f"  • {char_id}: {char_data['name']} ({char_data['series']})")

Available characters:
  • mikasa: Mikasa Ackerman (Attack on Titan)
  • sailor-moon: Sailor Moon (Sailor Moon)
  • wonder-woman: Wonder Woman (DC Comics)


In [4]:
# 🔑 Google Cloud Authentication
class ImagenGenerator:
    def __init__(self):
        self.project_id = PROJECT_ID
        self.location = LOCATION
        
        if not self.project_id:
            raise ValueError("GOOGLE_CLOUD_PROJECT_ID not set in .env file")
        
        # Initialize credentials
        try:
            self.credentials, _ = default()
            self.credentials.refresh(Request())
            print("✅ Google Cloud authentication successful!")
        except Exception as e:
            print(f"❌ Authentication failed: {e}")
            print("💡 Make sure your .env file is configured correctly")
            self.credentials = None
        
        # Imagen 4 Ultra endpoint
        self.endpoint = f"https://{self.location}-aiplatform.googleapis.com/v1/projects/{self.project_id}/locations/{self.location}/publishers/google/models/imagen-4.0-ultra-generate-001:predict"
    
    def generate_cosplay(self, image_path, character_id, output_name=None):
        """Generate cosplay transformation using Imagen 4 Ultra"""
        
        if not self.credentials:
            print("❌ No valid credentials - running in demo mode")
            return self._demo_mode(image_path, character_id, output_name)
        
        # Load and encode image
        with open(image_path, 'rb') as f:
            image_data = f.read()
        
        image_b64 = base64.b64encode(image_data).decode('utf-8')
        
        # Get character prompt
        if character_id not in CHARACTERS:
            raise ValueError(f"Character '{character_id}' not found")
        
        prompt = CHARACTERS[character_id]['prompt']
        character_name = CHARACTERS[character_id]['name']
        
        print(f"🎭 Generating {character_name} cosplay...")
        print(f"📸 Input: {image_path}")
        
        # Prepare API payload
        payload = {
            "instances": [{
                "prompt": prompt,
                "image": {
                    "bytesBase64Encoded": image_b64
                },
                "parameters": {
                    "sampleCount": 1,
                    "aspectRatio": "1:1",
                    "safetyFilterLevel": "block_some",
                    "personGeneration": "allow_adult",
                    "outputOptions": {
                        "compressionQuality": "lossless",
                        "mimeType": "image/png"
                    },
                    "editConfig": {
                        "editMode": "inpainting-replace",
                        "guidanceScale": 120,
                        "outputImageType": "EDITED_IMAGE"
                    },
                    "stylizationLevel": 100
                }
            }]
        }
        
        # Make API request
        headers = {
            "Authorization": f"Bearer {self.credentials.token}",
            "Content-Type": "application/json"
        }
        
        try:
            print("🚀 Calling Imagen 4 Ultra API...")
            response = requests.post(
                self.endpoint,
                headers=headers,
                json=payload,
                timeout=180
            )
            
            if response.status_code == 200:
                data = response.json()
                if "predictions" in data and data["predictions"]:
                    prediction = data["predictions"][0]
                    result_b64 = prediction.get("bytesBase64Encoded")
                    
                    if result_b64:
                        # Save result
                        return self._save_result(result_b64, character_id, character_name, output_name)
                    else:
                        print("❌ No image data in response")
                        return None
                else:
                    print("❌ No predictions in response")
                    return None
            else:
                print(f"❌ API Error {response.status_code}: {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Error: {e}")
            return None
    
    def _demo_mode(self, image_path, character_id, output_name):
        """Demo mode - creates a placeholder result"""
        character_name = CHARACTERS[character_id]['name']
        print(f"🎭 Demo Mode: Creating placeholder for {character_name}")
        
        # Create a simple demonstration image
        demo_img = Image.new('RGB', (512, 512), color='lightblue')
        
        # Save demo result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if not output_name:
            output_name = f"{character_id}_{timestamp}_demo"
        
        output_path = OUTPUT_FOLDER / f"{output_name}.png"
        demo_img.save(output_path)
        
        print(f"✅ Demo result saved: {output_path}")
        return output_path
    
    def _save_result(self, image_b64, character_id, character_name, output_name):
        """Save the generated image"""
        # Decode image
        image_data = base64.b64decode(image_b64)
        
        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if not output_name:
            output_name = f"{character_id}_{timestamp}"
        
        output_path = OUTPUT_FOLDER / f"{output_name}.png"
        
        # Save image
        with open(output_path, 'wb') as f:
            f.write(image_data)
        
        print(f"✅ {character_name} cosplay saved: {output_path}")
        return output_path

# Initialize generator
generator = ImagenGenerator()

❌ Authentication failed: ('invalid_scope: Invalid OAuth scope or ID token audience provided.', {'error': 'invalid_scope', 'error_description': 'Invalid OAuth scope or ID token audience provided.'})
💡 Make sure your .env file is configured correctly


In [ ]:
# 📋 List available model images
model_files = list(MODELS_FOLDER.glob("*.jpg")) + list(MODELS_FOLDER.glob("*.png")) + list(MODELS_FOLDER.glob("*.jpeg"))

print(f"📸 Found {len(model_files)} model images:")
for i, file_path in enumerate(model_files):
    print(f"  {i+1}. {file_path.name}")

if not model_files:
    print("⚠️  No images found in models/ folder. Please add some photos!")

In [ ]:
# 🎭 GENERATE COSPLAY - Single Image
# Configure this cell for your specific generation

# SETTINGS - Change these values
IMAGE_FILE = "model1.jpg"  # Change to your image filename
CHARACTER = "mikasa"       # Options: mikasa, sailor-moon, wonder-woman
OUTPUT_NAME = "mikasa_demo_1"  # Optional custom name

# Generate cosplay
image_path = MODELS_FOLDER / IMAGE_FILE

if image_path.exists():
    print(f"🎬 Starting cosplay generation...")
    print(f"📸 Model: {IMAGE_FILE}")
    print(f"🎭 Character: {CHARACTER}")
    print(f"⚡ Using Imagen 4 Ultra")
    print("-" * 50)
    
    start_time = time.time()
    result_path = generator.generate_cosplay(image_path, CHARACTER, OUTPUT_NAME)
    end_time = time.time()
    
    if result_path:
        print(f"⏱️  Generation time: {end_time - start_time:.2f} seconds")
        print(f"🎉 Success! Check your output folder: {result_path}")
    else:
        print("❌ Generation failed")
else:
    print(f"❌ Image not found: {image_path}")
    print(f"💡 Available images: {[f.name for f in model_files]}")

In [ ]:
# 🚀 BATCH PROCESSING - Generate All Characters for One Model
# Uncomment and configure this cell for batch processing

# BATCH_IMAGE = "model1.jpg"  # Change to your image filename
# 
# image_path = MODELS_FOLDER / BATCH_IMAGE
# if image_path.exists():
#     print(f"🎬 Batch processing: {BATCH_IMAGE}")
#     print("=" * 50)
#     
#     for char_id in CHARACTERS.keys():
#         print(f"\n🎭 Generating {CHARACTERS[char_id]['name']}...")
#         output_name = f"batch_{char_id}_{datetime.now().strftime('%H%M%S')}"
#         
#         result = generator.generate_cosplay(image_path, char_id, output_name)
#         if result:
#             print(f"✅ {char_id} completed")
#         else:
#             print(f"❌ {char_id} failed")
#         
#         time.sleep(2)  # Prevent rate limiting
#     
#     print("\n🎉 Batch processing complete!")
# else:
#     print(f"❌ Image not found: {image_path}")

In [ ]:
# 📊 Results Summary
output_files = list(OUTPUT_FOLDER.glob("*.png")) + list(OUTPUT_FOLDER.glob("*.jpg"))

print(f"📁 Generated {len(output_files)} cosplay images:")
for file_path in sorted(output_files):
    file_size = file_path.stat().st_size / 1024  # KB
    print(f"  • {file_path.name} ({file_size:.1f} KB)")

if output_files:
    print(f"\n🎉 All results saved in: {OUTPUT_FOLDER.absolute()}")
else:
    print("\n📝 No results yet. Run the generation cell above!")